# Introduction

[光度立体基础知识](https://www.cnblogs.com/linzzz98/articles/13622026.html)

介绍一下五种低维表现方法，这些方法都能从多幅数码相机中采集的不同光照方向的图像中提取合用的表现数据。

3I：该方法使用三幅图像，都使用翘角45°，倾斜角分别是0°，90°，180°。

Gradient: 这个方法使用立体光照技术提取表面梯度图和反照率图。

PTM: 这项研究成果应用了多项使纹理映射(Polynomial Texture Maps, PTM)。

Eigen3: 该方法是用了Eigen理论的前三张图。

Eigen6: 该方法与Eigen3方法类似，使用了Eigen理论的前六张图。


# SVD and image data matrix

在作业一中，我们知道SVD可以用来解决大多数的最小二乘法问题,不再详细介绍。

图像数据矩阵包括了各种光照方向下的所有图像数据。

假设每个图像有m个像素，我们采集了n幅图像。

我们用$i_{jk}$来表示第k幅图像中像素点j的亮度。

如果我们用二维坐标（x,y）来表示图像的位置，那么像素的编号j可以这样来表示:$j = (x-1)*w+y$

这里w是图像的宽度。即我们可以将所有图像中点的亮度表示在一个mxn的矩阵中，即：


$$I=\begin{bmatrix}
{i_{11}}&{i_{12}}&{\cdots}&{i_{1n}}\\
{i_{21}}&{i_{22}}&{\cdots}&{i_{2n}}\\
{\vdots}&{\vdots}&{\ddots}&{\vdots}\\
{i_{m1}}&{i_{m2}}&{\cdots}&{i_{mn}}\\
\end{bmatrix}
$$


在这个公式里面，每一列表示在某一特殊光照条件下一幅图像中的所有象素点亮度，而每一行则表示某一象素点在各种光照条件下的亮度。

我们将图像数据矩阵表示为乘积的形式，即$I=M_1M_2$

其中$M_1$和$M_2$是两个矩阵。$M_1$是我们想要的表面重光照表现数据矩阵。这样，如果我们知道了$M_2$,并且在假定了某一反射和重光照模型，我们可以通过SVD方法求出$M_1$。Gradient方法和PTM方法都属于这一类。对于3I方法来说，$M_1$只是一个原始图像数据矩阵$I$。如果我们不知道$M_2$或者我们不想假定任意一个反射和重光照模型，我们可以直接用SVD来分析图像数据矩阵I并且获取到$M_1$和$M_2$。后面将会提到基于Eigen的方法（Eigen3和Eigen6）就是这样做的。

重光照处理过程：表面表现数据矩阵$M_1$和特定光照条件向量$c$的乘积，即$i=M_1c$

$i$ 即为生成的图像数据向量，$i_1$,$i_2$,...,$i_m$ 表示其中的各个点的亮度值。

# Lambert model

在一个像素的位置上，朗伯反射公式表示为：$i(x,y)=λαn·l$

其中，i(x,y)表示坐标为（x,y）的像素点的亮度，λ是一个附加亮度信息，α是一个根据朗伯模型得到的反照率值，$n$是点（x,y）处的单位表面法向量，即
$n = (n_x,n_y,n_z)^T = (\frac{-p}{\sqrt[]{p^2+q^2+1}},\frac{-q}{\sqrt[]{p^2+q^2+1}},\frac{1}{\sqrt[]{p^2+q^2+1}})^T$

这其中 p 和 q 是表面高度图关于x和y的偏导数，即：

$p(x,y)=\frac{∂s(x,y)}{∂x}$ , $q(x,y)=\frac{∂s(x,y)}{∂y}$

s(x,y)是表面高度函数。

$l$是点（x,y）处的单位光照向量，即

$l = (l_x,l_y,l_z)^T = (cosτsinσ,sinτsinσ,cos σ)^T$

这里，σ是光源的翘角slant，τ是倾斜角tilt。

由于假设附加的亮度λ信息是一个常量，因此完全可以将它合并在反照率α。我们用ρ来表示这两个量的乘积，ρ=λα。

图像数据表示为 $I = ANL$

其中，$A$是表面的反照率矩阵
$$A=\begin{bmatrix}
 ρ_1&&&0\\
 &ρ_2&&\\
 &&\ddots&\\
 0&&&ρ_m\\
\end{bmatrix}$$
$N$是表面法向量矩阵
$$N=(n_1,n_2,...,n_m)^T=\begin{bmatrix}
 n_{1x}&n_{1y}&n_{1z}\\
 n_{2x}&n_{2y}&n_{2z}\\
 \vdots&\vdots&\vdots\\
 n_{mx}&n_{my}&n_{mz}\\
\end{bmatrix}
$$
$L$是光照矩阵
$$N=(l_1,l_2,...,l_m)=\begin{bmatrix}
 l_{1x}&l_{2x}&\cdots&l_{mx}\\
 l_{1y}&l_{2y}&\cdots&l_{my}\\
 l_{1z}&l_{2z}&\cdots&l_{mz}\\
\end{bmatrix}
$$

进一步定义一个新的矩阵$N_a$，$N_a=AN$

即$I = N_aL$

# 3I method

3I方法由三幅光度学图像线性合并生成新的图像。

Shashua的研究发现，表面凸凹不平的物体图像的表现可以由满足朗伯反射模型假设的三幅图像进行线性合成得到。我们称这种方法为3I。

这三幅图像是分别在光源位置不同的情况下得到的。这三幅图像称为“光度测定图”。这样，根据$i=M_1c$，我们只需要决定包含线性合成系数的向量c就可以了。根据$I = N_aL$，并计算好反向光照矩阵即可。

我们得到了三个非相关的光照向量，光照矩阵可以这样表达
$$L=(l_1,l_2,l_3)=\begin{bmatrix}
l_{1x}&l_{2x}&l_{3x}\\
l_{1y}&l_{2y}&l_{3y}\\
l_{1z}&l_{2z}&l_{3z}\\
\end{bmatrix}$$

同样，三幅图像的图像数据矩阵为 
$$I=\begin{bmatrix} 
l_{11}&l_{12}&l_{13}\\
l_{21}&l_{22}&l_{23}\\
\vdots&\vdots&\vdots \\
l_{m1}&l_{m2}&l_{m3}\\
\end{bmatrix}$$

则 $N_a=IL^{-1}$

重光照：任意给定光照方向可得到相应的光照向量$l = (l_x,l_y,l_z)^T = (cosτsinσ,sinτsinσ,cos σ)^T$

即可得到生成的图像的个像素点亮度，表示为：$i=IL^{-1}l$


而根据$i=M_1c$，我们又可以得到$c=L^{-1}l$,

即$i=M_1c=Ic$

我们就可以得到结论：给定光照方向生成的新的图像是由三幅已知图像线性组合得到的，其中的$c$称为系数向量。


In [ ]:
#todo 

# gradient method

Gradient方法使用表面梯度和反照率图作为表面表现数据进行重光照。

根据朗伯模型定律，表面梯度和反照率图可以作为三维表面纹理的表现数据进行重光照，我们称这种方法为Gradient。3I使用三幅图像来建立符合朗伯模型的表面梯度和反照率图。过多的图导致系统成为一个过约束系统，无解，但是可以通过最小平方技术（比如SVD)来得到相对来说更加精确的解。

在$I=N_aL$中，$I$为一个mxn的矩阵，n为采用的图片数；$L$为光照矩阵，是一个已知的3xm的矩阵。

根据$I=N_aL$和$I=M_1M_2$得到，$M_1=N_a$且$M_2=L$

$N_a$包含了表面梯度和反照率信息，未知，且用SVD求解释非常繁琐。

因此，我们首先将光照矩阵分解：

$L=U_LW_L{V_L}^T$

即有：$N_a=IL^{-1}=IV_LW_L^{-1}U_L^T$

对$N_a$光照，就可以得到任意光照条件下的新图像：

$i=N_al$


In [ ]:
# todo

# PTM method

PTM方法使用多项式纹理映射作为表面纹理表示方式。Malzbender等人提出了一种亮度表示模型, 用两度向量的二次多项式来表示因自己产生的阴影和反射导致的复杂纹理表示形式。它也是基于朗伯模型的假设，并且使用了单位光照向量的前两项来组成了一个新的六维向量，即

$l_{ptm}=(l_x^2,l_y^2,l_xl_y,l_x,l_y,1)^T=(cosτ^2sinσ^2,sinτ^2sinσ^2,cosτsinτsinσ^2，cosτsinσ，sinτsinσ，1)^T$

则图像数据为$I=A_{ptm}L_{ptm}$

其中，$A_{ptm}$为PTM矩阵，矩阵的每一行表示在每个像素位置处根据亮度模型的六个系数，$L_{ptm}$为光照矩阵，分别等价于$M_1$和$M_2$

$$A_{ptm}=\begin{bmatrix}
 a_{11}&a_{12}&a_{13}&a_{14}&a_{15}&a_{16}\\
 a_{21}&a_{22}&a_{23}&a_{24}&a_{25}&a_{26}\\
 \vdots&\vdots&\vdots&\vdots&\vdots&\vdots\\
 a_{m1}&a_{m2}&a_{m3}&a_{m4}&a_{m5}&a_{m6}\\
\end{bmatrix}
$$

$$L_{ptm}=\begin{bmatrix}
 1&1&\cdots&1\\
 l_{y1}&l_{y2}&\cdots&l_{yn}\\
 l_{x1}&l_{x2}&\cdots&l_{xn}\\
 l_{x1}l_{y1}&l_{x2}l_{y2}&\cdots&l_{xn}l_{yn}\\
 l_{y1}^2&l_{y2}^2&\cdots&l_{yn}^2\\
 l_{x1}^2&l_{x2}^2&\cdots&l_{xn}^2
\end{bmatrix}
$$


$I$和$L_{PTM}$都已知，可以使用SVD来求解得到$A_{PTM}$

重光照，新生成的图像为：

$i=M_1c=A_{ptm}L_{PTM}$

In [ ]:
#todo

# Eigen method

Eigen模型是许多研究人员使用的一种特殊的处理各种光照的模型。这些方法的优势在于不需要任何表面反射模型的假设。Epstein等人在实验的基础上提出了一种有效的使用五幅图像为基础来合成任意光照条件下的物体纹理。他们的断定这项研究成果可以在物体有光滑顶部、光滑的尖起、微弱的阴影的情况下，模拟朗伯表面模型。当然，由于镜面反射和复杂的表面几何形状都增加了所需的图像数目。

Eigen3算法在Eigen空间上采用三幅图象，Eigen6算法在Eigen空间上采用6幅图象的来重建三维表面纹理。Eigen3可以实现符合朗伯模型的三维表面纹理合成，而Eigen6则可以实现带有特定镜面反射和阴影的表面纹理的合成。应用SVD来生成Eigen空间下的数据，图像的数据矩阵表示形式为:

$I=U_IW_IV_I^T$

其中矩阵$U_I$中的每一列即是矩阵$W_I$的奇异值相应的Eigen向量。$U_I$用来构建基于Eigen的图像，而$V_I^T$包含了线性合成时用到的系数。

将$W_I$表示成$W_I=diag(w_1,w_2,...,w_n)$

矩阵$W_I$的一个重要的属性就是其奇异值非常少。前面的几个特征向量就已占据了整个图像数据矩阵I中超过99%的信息。因此将$W_I$近似为

$\hat{W_I}=diag(w_1,w_2,...,w_k,0,...,0)$
其中k是希望保留的特征值的个数。

然后我们就得到了一个近似的图像数据矩阵$I$,为$\hat{I}=U_I\hat{W_I}V_I^T$

由于$I=M_1M_2$,

我们得到$M_1=U_I\hat{W_I}$, $M_2=V_I^T$

重新令$M_1$是mxk的矩阵，其中$U_I\hat{W_I}$后面的n-k列全部为0。

类似的，我们重写$M_2$是kxn的矩阵，里面只保留了$V_I^T$中的前k行数据，因为\hat{W_I}的后面n-k列全部为0。

通过如此操作，我们得到了在特征向量空间的一系列的k-基图像数据，这些图像称之为Eigen base image。使用3幅Eigen base image作为表现数据时，就是Eigen 3, 6幅为Eigen 6。矩阵$M_2$提供了根据这些Eigen base image图像得到原始图像$I$数据之间的线性合成系数。

因此，$I=(i_1,i_2,...,i_m)=M_1M_2$

In [ ]:
#todo